<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Prerequisites" data-toc-modified-id="Prerequisites-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Prerequisites</a></span></li><li><span><a href="#Argo-deployment" data-toc-modified-id="Argo-deployment-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Argo deployment</a></span></li><li><span><a href="#Client" data-toc-modified-id="Client-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Client</a></span></li><li><span><a href="#Workflows" data-toc-modified-id="Workflows-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Workflows</a></span><ul class="toc-item"><li><span><a href="#Create" data-toc-modified-id="Create-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Create</a></span></li><li><span><a href="#Get" data-toc-modified-id="Get-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Get</a></span></li><li><span><a href="#Patch" data-toc-modified-id="Patch-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Patch</a></span></li><li><span><a href="#Watch" data-toc-modified-id="Watch-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Watch</a></span></li><li><span><a href="#Delete" data-toc-modified-id="Delete-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Delete</a></span></li></ul></li></ul></div>

## Requirements <a class="tocSkip">

In [ ]:
%dep clear

In [ ]:
%%requirements

[[source]]
name = "pypi"
url = "https://pypi.org/simple"
verify_ssl = true

[packages]
argo-workflows = ">=1.0"

[requires]
python_version = "3.6"

In [ ]:
%dep ensure --engine=pipenv argo-workflows

## Prerequisites

Running Kubernetes/OpenShift cluster.

See for example [minishift](https://www.okd.io/minishift/) or [minikube](https://kubernetes.io/docs/setup/learning-environment/minikube/)

## Argo deployment

To deploy argo into the namespace, make use of the prepared [Provisioning Playbooks](../playbooks/provisioning.yaml):

Make sure you have Argo CRDs present in the cluster and run the playbook:

```
ansible-playbook playbooks/provisioning.yaml --ask-vault-pass
```

The default namespace assumed is `argo`, you can change it in any point by running

```
ansible-playbook playbooks/provisioning.yaml --e namespace=<custom-namespace> --ask-vault-pass
```

or by modifying the Playbook.

## Client

In [ ]:
from argo.workflows.client import ApiClient
from argo.workflows.client import V1alpha1Api

from argo.workflows.config import load_kube_config

In [ ]:
load_kube_config()  # loads local configuration from ~/.kube/config

> For in-cluster configuration, use the `load_incluster_config()` or create a new client manually with `new_client_from_config()` functions

Set the namespace the argo is deployed in

In [ ]:
namespace = "argo"

Instantiate the API

In [ ]:
api = ApiClient()
v1alpha1 = V1alpha1Api(api)

And to list Workflows from a namespace (say argo):

In [ ]:
wfs = v1alpha1.list_namespaced_workflows(namespace)
wfs

## Workflows

Load an example workflow from the [examples](../examples/) folder

In [ ]:
import yaml
from pathlib import Path

In [ ]:
wf = yaml.safe_load(Path('../examples/inspection-batch.yaml').read_text())
wf

### Create

Create a workflow from the yaml specification

In [ ]:
wf = v1alpha1.create_namespaced_workflow(namespace, wf)
wf

### Get

In [ ]:
wf = v1alpha1.get_namespaced_workflow(namespace, wf)
wf

### Patch

Let's say that we want to patch the workflow now to run with `batch_size = 1`

In [ ]:
patch = [{
    "op": "replace",
    "path": "/spec/arguments/parameters/0/value",
    "value": '1'
}]

v1alpha1.patch_namespaced_workflow(namespace, wf["metadata"]["name"], patch)

### Watch

Watching the workflow resources works the way Kubernetes users are used to

In [ ]:
from argo.workflows import watch

w = watch.Watch()
for event in w.stream(v1alpha1.list_namespaced_workflows, namespace=namespace):
    print(event)
    break

### Delete

And delete the workflow

In [ ]:
v1alpha1.delete_namespaced_workflow(namespace, wf["metadata"]["name"])